In [ ]:
!pip install transformers

     |████████████████████████████████| 2.0MB 8.5MB/s 
     |████████████████████████████████| 3.2MB 49.7MB/s 
     |████████████████████████████████| 890kB 55.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=5bd2962ef17a16c5b7b91a75a76e609106d15c95469f631f5689c0c63b7cbe8b
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
# Importing stock ml libraries
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn import preprocessing
from sklearn import model_selection
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertModel, BertConfig

In [ ]:
print(np.__version__)
print(pd.__version__)
print(torch.__version__)
print(transformers.__version__)
print(np.__version__)
print(np.__version__)

1.19.5
1.1.5
1.8.0+cu101
4.4.2
1.19.5
1.19.5


In [ ]:

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [ ]:
 
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
df17 = pd.read_csv('/content/drive/My Drive/MTLBACategory.csv')
df17 = df17.drop(['aspectCategories'], axis=1)

df29 = df17.reset_index(drop=True)
df29.shape

(800, 6)

In [ ]:
df29.head(10)

,text,service,food,miscellanous,price,ambience
0,The bread is top notch as well.,0,1,0,0,0
1,I have to say they have one of the fastest del...,1,0,0,0,0
2,Food is always fresh and hot- ready to eat!,0,1,0,0,0
3,Did I mention that the coffee is OUTSTANDING?,0,1,0,0,0
4,"Certainly not the best sushi in New York, howe...",0,1,0,0,1
5,"I trust the people at Go Sushi, it never disap...",0,0,1,0,0
6,"Straight-forward, no surprises, very decent Ja...",0,1,0,0,0
7,"BEST spicy tuna roll, great asian salad.",0,1,0,0,0
8,Try the rose roll (not on menu).,0,1,0,0,0
9,"I love the drinks, esp lychee martini, and the...",0,1,0,0,0


In [ ]:
df17['list'] = df17[df17.columns[1:]].values.tolist()
#final = final.rename(columns={'Sentence': 'text'})
df27 = df17[['text', 'list']].copy()
df27.head(50)

,text,list
0,The bread is top notch as well.,"[0, 1, 0, 0, 0]"
1,I have to say they have one of the fastest del...,"[1, 0, 0, 0, 0]"
2,Food is always fresh and hot- ready to eat!,"[0, 1, 0, 0, 0]"
3,Did I mention that the coffee is OUTSTANDING?,"[0, 1, 0, 0, 0]"
4,"Certainly not the best sushi in New York, howe...","[0, 1, 0, 0, 1]"
5,"I trust the people at Go Sushi, it never disap...","[0, 0, 1, 0, 0]"
6,"Straight-forward, no surprises, very decent Ja...","[0, 1, 0, 0, 0]"
7,"BEST spicy tuna roll, great asian salad.","[0, 1, 0, 0, 0]"
8,Try the rose roll (not on menu).,"[0, 1, 0, 0, 0]"
9,"I love the drinks, esp lychee martini, and the...","[0, 1, 0, 0, 0]"


In [ ]:
df27.shape

(800, 2)

In [ ]:
df18 = pd.read_csv('/content/drive/My Drive/sentence (1).txt', sep='delimiter', names=["Sentence1"])
df19 = pd.read_csv('/content/drive/My Drive/target (1).txt', sep='delimiter', names=["target1"])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [ ]:
df18.shape


(800, 1)

In [ ]:
frame = [df18, df19]

df20 = pd.concat(frame, axis=1)

df20.head(50)

,Sentence1,target1
0,the bread is top notch as well,0 1 0 0 0 0 0
1,i have to say they have one of the fastest del...,0 0 0 0 0 0 0 0 0 0 1 2 0 0 0
2,food is always fresh and hot- ready to eat !,1 0 0 0 0 0 0 0 0 0
3,did i mention that the coffee is outstanding,0 0 0 0 0 1 0 0
4,"certainly not the best sushi in new york , how...",0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0
5,"i trust the people at go sushi , it never disa...",0 0 0 1 0 0 0 0 0 0 0
6,"straight-forward , no surprises , very decent ...",0 0 0 0 0 0 0 1 2
7,"best spicy tuna roll , great asian salad",0 1 2 2 0 0 1 2
8,try the rose roll ( not on menu ),0 0 1 2 0 0 0 1 0
9,"i love the drinks , esp lychee martini , and t...",0 0 0 1 0 0 1 2 0 0 0 1 0 0 0 0


In [ ]:
u = df20.Sentence1.str.split(expand=True).stack()
v = df20.target1.str.split(expand=True).stack()

df21 = pd.DataFrame({
        'Sentence': u.index.get_level_values(0) + 1, 
        'Word': u.values, 
        'target': v.values,
        #'Entity': u.map(dict(zip(df.value, df.entity))).fillna('Object').values
    })

In [ ]:
def te_sentiment(rating):
  rating = int(rating)
  if rating == 2:
    return 'I'
  elif rating == 1:
    return 'B'
  else:
    return 'O'
df21['target'] = df21.target.apply(te_sentiment)


In [ ]:
df21.head(50)

,Sentence,Word,target
0,1,the,O
1,1,bread,B
2,1,is,O
3,1,top,O
4,1,notch,O
5,1,as,O
6,1,well,O
7,2,i,O
8,2,have,O
9,2,to,O


In [ ]:
def te_target(rating):
  rating = str(rating)
  if rating == 'I':
    return 3
  elif rating == 'B':
    return 2
  else:
    return 1
df21['target'] = df21.target.apply(te_target)

In [ ]:
df14 = pd.DataFrame()
df14['temp'] = 0
df14['temp'] = df14['temp'].astype('object')
df15 = pd.DataFrame()
df15['target'] = 0
df15['target'] = df15['target'].astype('object')

In [ ]:

def process_data(df, df4, df5):

    enc_tag = preprocessing.LabelEncoder() 

    #df.loc[:, "target"] = enc_tag.fit_transform(df["target"])

    sentences = df.groupby("Sentence")["Word"].apply(list).values
    print(len(sentences))
    counter = 0
    for i in sentences:
      print(i)
      df4.at[counter, 'temp'] = np.array(i)
      counter = counter + 1
    tag = df.groupby("Sentence")["target"].apply(list).values
    c = 0
    for i in tag:
      print(i)
      df5.at[c, 'target'] = np.asarray(i)
      c = c + 1
    return sentences, tag, enc_tag, df4, df5

In [ ]:
sentences_te, tag_te, enc_tag_te, df14, df15 = process_data(df21, df14, df15) 


800
['the', 'bread', 'is', 'top', 'notch', 'as', 'well']
['i', 'have', 'to', 'say', 'they', 'have', 'one', 'of', 'the', 'fastest', 'delivery', 'times', 'in', 'the', 'city']
['food', 'is', 'always', 'fresh', 'and', 'hot-', 'ready', 'to', 'eat', '!']
['did', 'i', 'mention', 'that', 'the', 'coffee', 'is', 'outstanding']
['certainly', 'not', 'the', 'best', 'sushi', 'in', 'new', 'york', ',', 'however', ',', 'it', 'is', 'always', 'fresh', ',', 'and', 'the', 'place', 'is', 'very', 'clean', ',', 'sterile']
['i', 'trust', 'the', 'people', 'at', 'go', 'sushi', ',', 'it', 'never', 'disappoints']
['straight-forward', ',', 'no', 'surprises', ',', 'very', 'decent', 'japanese', 'food']
['best', 'spicy', 'tuna', 'roll', ',', 'great', 'asian', 'salad']
['try', 'the', 'rose', 'roll', '(', 'not', 'on', 'menu', ')']
['i', 'love', 'the', 'drinks', ',', 'esp', 'lychee', 'martini', ',', 'and', 'the', 'food', 'is', 'also', 'very', 'good']
['in', 'fact', ',', 'this', 'was', 'not', 'a', 'nicoise', 'salad', 'and

In [ ]:
frames = [df14, df15, df27]

final_te = pd.concat(frames, axis=1)
final_te.head(20)

,temp,target,text,list
0,"[the, bread, is, top, notch, as, well]","[1, 2, 1, 1, 1, 1, 1]",The bread is top notch as well.,"[0, 1, 0, 0, 0]"
1,"[i, have, to, say, they, have, one, of, the, f...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 3, 1, 1, 1]",I have to say they have one of the fastest del...,"[1, 0, 0, 0, 0]"
2,"[food, is, always, fresh, and, hot-, ready, to...","[2, 1, 1, 1, 1, 1, 1, 1, 1, 1]",Food is always fresh and hot- ready to eat!,"[0, 1, 0, 0, 0]"
3,"[did, i, mention, that, the, coffee, is, outst...","[1, 1, 1, 1, 1, 2, 1, 1]",Did I mention that the coffee is OUTSTANDING?,"[0, 1, 0, 0, 0]"
4,"[certainly, not, the, best, sushi, in, new, yo...","[1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","Certainly not the best sushi in New York, howe...","[0, 1, 0, 0, 1]"
5,"[i, trust, the, people, at, go, sushi, ,, it, ...","[1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1]","I trust the people at Go Sushi, it never disap...","[0, 0, 1, 0, 0]"
6,"[straight-forward, ,, no, surprises, ,, very, ...","[1, 1, 1, 1, 1, 1, 1, 2, 3]","Straight-forward, no surprises, very decent Ja...","[0, 1, 0, 0, 0]"
7,"[best, spicy, tuna, roll, ,, great, asian, salad]","[1, 2, 3, 3, 1, 1, 2, 3]","BEST spicy tuna roll, great asian salad.","[0, 1, 0, 0, 0]"
8,"[try, the, rose, roll, (, not, on, menu, )]","[1, 1, 2, 3, 1, 1, 1, 2, 1]",Try the rose roll (not on menu).,"[0, 1, 0, 0, 0]"
9,"[i, love, the, drinks, ,, esp, lychee, martini...","[1, 1, 1, 2, 1, 1, 2, 3, 1, 1, 1, 2, 1, 1, 1, 1]","I love the drinks, esp lychee martini, and the...","[0, 1, 0, 0, 0]"


In [ ]:
final_te.iloc[798]

temp      [although, the, restaurant, itself, is, nice, ...
target        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2]
text      Although the restaurant itself is nice, I pref...
list                                        [0, 1, 0, 0, 1]
Name: 798, dtype: object

In [ ]:
df1 = pd.read_csv('/content/drive/My Drive/sentence.txt', sep='delimiter', names=["Sentence"])
df2 = pd.read_csv('/content/drive/My Drive/target.txt', sep='delimiter', names=["target"])
df3 = pd.read_csv('/content/drive/My Drive/semeval2014.csv')

frames = [df1, df2]

df = pd.concat(frames, axis=1)

df.head(50)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


,Sentence,target
0,but the staff was so horrible to us,0 0 1 0 0 0 0 0
1,"to be completely fair , the only redeeming fac...",0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 ...
2,"the food is uniformly exceptional , with a ver...",0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 ...
3,where gabriela personaly greets you and recomm...,0 0 0 0 0 0 0 0 0 0 0
4,"for those that go once and do n't enjoy it , a...",0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
5,"not only was the food outstanding , but the li...",0 0 0 0 1 0 0 0 0 0 0 1 0 0 0
6,it is very overpriced and not very tasty,0 0 0 0 0 0 0 0
7,our agreed favorite is the orrechiete with sau...,0 0 0 0 0 1 2 2 2 2 0 0 0 1 0 0 0 0 0 0 1 0 0 ...
8,the bagels have an outstanding taste with a te...,0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
9,nevertheless the food itself is pretty good,0 0 1 0 0 0 0


In [ ]:
u = df.Sentence.str.split(expand=True).stack()
v = df.target.str.split(expand=True).stack()

df7 = pd.DataFrame({
        'Sentence': u.index.get_level_values(0) + 1, 
        'Word': u.values, 
        'target': v.values,
        #'Entity': u.map(dict(zip(df.value, df.entity))).fillna('Object').values
    })

In [ ]:
def to_sentiment(rating):
  rating = int(rating)
  if rating == 2:
    return 'I'
  elif rating == 1:
    return 'B'
  else:
    return 'O'
df7['target'] = df7.target.apply(to_sentiment)


In [ ]:
df7.head(50)

,Sentence,Word,target
0,1,but,O
1,1,the,O
2,1,staff,B
3,1,was,O
4,1,so,O
5,1,horrible,O
6,1,to,O
7,1,us,O
8,2,to,O
9,2,be,O


In [ ]:
df4 = pd.DataFrame()
df4['temp'] = 0
df4['temp'] = df4['temp'].astype('object')
df5 = pd.DataFrame()
df5['target'] = 0
df5['target'] = df5['target'].astype('object')


In [ ]:
def to_target(rating):
  rating = str(rating)
  if rating == 'I':
    return 3
  elif rating == 'B':
    return 2
  else:
    return 1
df7['target'] = df7.target.apply(to_target)


In [ ]:

def process_data(df, df4, df5):

    enc_tag = preprocessing.LabelEncoder() 

    #df.loc[:, "target"] = enc_tag.fit_transform(df["target"])

    sentences = df.groupby("Sentence")["Word"].apply(list).values
    print(len(sentences))
    counter = 0
    for i in sentences:
      print(i)
      df4.at[counter, 'temp'] = np.array(i)
      counter = counter + 1
    tag = df.groupby("Sentence")["target"].apply(list).values
    c = 0
    for i in tag:
      print(i)
      df5.at[c, 'target'] = np.asarray(i)
      c = c + 1
    return sentences, tag, enc_tag, df4, df5


In [ ]:
sentences, tag, enc_tag, df4, df5 = process_data(df7, df4, df5) 
print(sentences)
print("***********************")
print(tag)
print("***********************")


In [ ]:

for t in tag:
  count = 0
  if(t==2):
    count = count+1
print(count)

0


In [ ]:
df4.shape
df4.head(5)

,temp
0,"[but, the, staff, was, so, horrible, to, us]"
1,"[to, be, completely, fair, ,, the, only, redee..."
2,"[the, food, is, uniformly, exceptional, ,, wit..."
3,"[where, gabriela, personaly, greets, you, and,..."
4,"[for, those, that, go, once, and, do, n't, enj..."


In [ ]:
import joblib

meta_data = {
        "enc_tag": enc_tag
    }

joblib.dump(meta_data, "meta.bin")

#num_tag = len(list(enc_tag.classes_))
num_tag = 4
print(num_tag)

4


In [ ]:
#df = pd.read_csv('/content/drive/My Drive/semeval2014.csv')

df3['list'] = df3[df3.columns[1:]].values.tolist()
#final = final.rename(columns={'Sentence': 'text'})
new_df = df3[['text', 'list']].copy()
new_df.head(10)

,text,list
0,but the staff was so horrible to us,"[1, 0, 0, 0, 0]"
1,to be completely fair the only redeeming facto...,"[0, 1, 1, 0, 0]"
2,the food is uniformly exceptional with a very ...,"[0, 1, 0, 0, 0]"
3,where gabriela personaly greets you and recomm...,"[1, 0, 0, 0, 0]"
4,for those that go once and dont enjoy it all i...,"[0, 0, 1, 0, 0]"
5,not only was the food outstanding but the litt...,"[1, 1, 0, 0, 0]"
6,it is very overpriced and not very tasty,"[0, 1, 0, 1, 0]"
7,our agreed favorite is the orrechiete with sau...,"[1, 1, 0, 0, 0]"
8,the bagels have an outstanding taste with a te...,"[0, 1, 0, 0, 0]"
9,nevertheless the food itself is pretty good,"[0, 1, 0, 0, 0]"


In [ ]:
frames = [df4, df5, new_df]

final = pd.concat(frames, axis=1)
final.head(20)

,temp,target,text,list
0,"[but, the, staff, was, so, horrible, to, us]","[1, 1, 2, 1, 1, 1, 1, 1]",but the staff was so horrible to us,"[1, 0, 0, 0, 0]"
1,"[to, be, completely, fair, ,, the, only, redee...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, ...",to be completely fair the only redeeming facto...,"[0, 1, 1, 0, 0]"
2,"[the, food, is, uniformly, exceptional, ,, wit...","[1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, ...",the food is uniformly exceptional with a very ...,"[0, 1, 0, 0, 0]"
3,"[where, gabriela, personaly, greets, you, and,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",where gabriela personaly greets you and recomm...,"[1, 0, 0, 0, 0]"
4,"[for, those, that, go, once, and, do, n't, enj...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",for those that go once and dont enjoy it all i...,"[0, 0, 1, 0, 0]"
5,"[not, only, was, the, food, outstanding, ,, bu...","[1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1]",not only was the food outstanding but the litt...,"[1, 1, 0, 0, 0]"
6,"[it, is, very, overpriced, and, not, very, tasty]","[1, 1, 1, 1, 1, 1, 1, 1]",it is very overpriced and not very tasty,"[0, 1, 0, 1, 0]"
7,"[our, agreed, favorite, is, the, orrechiete, w...","[1, 1, 1, 1, 1, 2, 3, 3, 3, 3, 1, 1, 1, 2, 1, ...",our agreed favorite is the orrechiete with sau...,"[1, 1, 0, 0, 0]"
8,"[the, bagels, have, an, outstanding, taste, wi...","[1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",the bagels have an outstanding taste with a te...,"[0, 1, 0, 0, 0]"
9,"[nevertheless, the, food, itself, is, pretty, ...","[1, 1, 2, 1, 1, 1, 1]",nevertheless the food itself is pretty good,"[0, 1, 0, 0, 0]"


In [ ]:
final.shape

(3044, 4)

In [ ]:
final.head(50)

,temp,target,text,list
0,"[but, the, staff, was, so, horrible, to, us]","[1, 1, 2, 1, 1, 1, 1, 1]",but the staff was so horrible to us,"[1, 0, 0, 0, 0]"
1,"[to, be, completely, fair, ,, the, only, redee...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, ...",to be completely fair the only redeeming facto...,"[0, 1, 1, 0, 0]"
2,"[the, food, is, uniformly, exceptional, ,, wit...","[1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, ...",the food is uniformly exceptional with a very ...,"[0, 1, 0, 0, 0]"
3,"[where, gabriela, personaly, greets, you, and,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",where gabriela personaly greets you and recomm...,"[1, 0, 0, 0, 0]"
4,"[for, those, that, go, once, and, do, n't, enj...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",for those that go once and dont enjoy it all i...,"[0, 0, 1, 0, 0]"
5,"[not, only, was, the, food, outstanding, ,, bu...","[1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1]",not only was the food outstanding but the litt...,"[1, 1, 0, 0, 0]"
6,"[it, is, very, overpriced, and, not, very, tasty]","[1, 1, 1, 1, 1, 1, 1, 1]",it is very overpriced and not very tasty,"[0, 1, 0, 1, 0]"
7,"[our, agreed, favorite, is, the, orrechiete, w...","[1, 1, 1, 1, 1, 2, 3, 3, 3, 3, 1, 1, 1, 2, 1, ...",our agreed favorite is the orrechiete with sau...,"[1, 1, 0, 0, 0]"
8,"[the, bagels, have, an, outstanding, taste, wi...","[1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",the bagels have an outstanding taste with a te...,"[0, 1, 0, 0, 0]"
9,"[nevertheless, the, food, itself, is, pretty, ...","[1, 1, 2, 1, 1, 1, 1]",nevertheless the food itself is pretty good,"[0, 1, 0, 0, 0]"


In [ ]:
# Sections of config

# Defining some key variables that will be used later on in the training
MAX_LEN = 100
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 4
EPOCHS = 5
#epochs = 3
LEARNING_RATE = 2e-05
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',return_dict=False)

In [ ]:
class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.temp
        
        self.tag = self.data.target
        self.targets = self.data.list
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = self.text[index]
        tags = self.tag[index]
        targets = self.targets[index]

        ids = []
        target_tag =[]
        #target =[]

        for i, s in enumerate(text):
            inputs = self.tokenizer.encode(
                s,
                add_special_tokens=False
            )
            # abhishek: ab ##hi ##sh ##ek
            input_len = len(inputs)
            ids.extend(inputs)
            target_tag.extend([tags[i]] * input_len)

        ids = ids[:MAX_LEN - 2]
        target_tag = target_tag[:MAX_LEN - 2]

        ids = [101] + ids + [102]
        target_tag = [1] + target_tag + [1]

        mask = [1] * len(ids)
        token_type_ids = [0] * len(ids)

        padding_len = MAX_LEN - len(ids)

        ids = ids + ([0] * padding_len)
        mask = mask + ([0] * padding_len)
        token_type_ids = token_type_ids + ([0] * padding_len)
        target_tag = target_tag + ([0] * padding_len)

        #marked_text = "[CLS] " + text + " [SEP]"
        #tokenized_text = self.tokenizer.encode(text)
        '''
        print(text)
        print("***********************")
        #print(tokenized_text)
        #print("***********************")
        print(tags)
        print("***********************")
        print(ids)
        print("***********************")
        print(mask)
        print("***********************")
        print(token_type_ids)
        print("***********************")
        print(target_tag)
        print("***********************")
        print(targets)
        print("***********************")
        '''

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            "target_tag": torch.tensor(target_tag, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [ ]:
# Creating the dataset and dataloader for the neural network

#train_size = 0.8
#train_dataset=final.sample(frac=train_size,random_state=200)
#test_dataset=final.drop(train_dataset.index).reset_index(drop=True)
#train_dataset = train_dataset.reset_index(drop=True)


train_size = 1.00
test_size = 1.0
train_dataset=final.sample(frac=train_size,random_state=200)
test_dataset=final_te.sample(frac=test_size,random_state=200)
train_dataset = train_dataset.reset_index(drop=True)

print("FULL Dataset: {}".format(final.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = CustomDataset(train_dataset, tokenizer, MAX_LEN)
testing_set = CustomDataset(test_dataset, tokenizer, MAX_LEN)

FULL Dataset: (3044, 4)
TRAIN Dataset: (3044, 4)
TEST Dataset: (800, 4)


In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [ ]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model. 

fchidden = 256
hiddendim_lstm = 256
embeddim = 768
numlayers = 12
numclasses = 4
num_heads = 2

class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.numclasses = numclasses
        self.embeddim = embeddim
        self.numlayers = numlayers
        self.hiddendim_lstm = hiddendim_lstm
        self.l1 = transformers.BertModel.from_pretrained('bert-base-uncased', return_dict=False)
        self.l2 = torch.nn.Dropout(0.3)
        self.l3 = torch.nn.Dropout(0.3)
        self.l4 = torch.nn.Linear(768, 4)
        self.l5 = torch.nn.Linear(768, 5)
    
    def forward(self, ids, mask, token_type_ids):
        o1, pooled_output = self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids)
        #print(output_2.shape)
        output_2 = self.l2(o1)
        #print(output_2.shape)
        output_3 = self.l3(pooled_output)
        #output_2 = self.lstm(output_2, None)
        #output_2 = self.conv(output_2)
        #output_2 = self.conv(output_2)
        #print(output_2.shape)
        output = self.l4(output_2)
        
        out = self.l5(output_3)
        return output, out



class Bert_LSTM(torch.nn.Module):
    def __init__(self, numclasses):
        super(Bert_LSTM, self).__init__()
        self.numclasses = numclasses
        self.embeddim = embeddim
        self.numlayers = numlayers
        self.hiddendim_lstm = hiddendim_lstm
        self.dropout = torch.nn.Dropout(0.1)

        #self.lstm = torch.nn.LSTM(self.embeddim, self.hiddendim_lstm*2, bidirectional = True, batch_first=False)
        #self.conv = torch.nn.Conv1d( 100, self.hiddendim_lstm, 3, stride=2)
        self.bert = transformers.BertModel.from_pretrained('bert-base-uncased', return_dict=False)
        print("BERT Model Loaded")
        self.lstm = torch.nn.LSTM(self.embeddim, self.hiddendim_lstm, bidirectional = False, batch_first=False)
        #self.lstm = torch.nn.GRU(self.embeddim, self.hiddendim_lstm, bidirectional = True, batch_first=False) # noqa
        #self.attention = torch.nn.MultiheadAttention(self.hiddendim_lstm*2, num_heads = 2)
        self.fc = torch.nn.Linear(self.hiddendim_lstm, 4)
        self.fc1 = torch.nn.Linear(self.hiddendim_lstm, 5)

    def forward(self, ids, mask, token_type_ids):
      last_hidden_state, pooled_output = self.bert(ids, attention_mask = mask, token_type_ids = token_type_ids)
      #print(last_hidden_state.shape)
      #print(pooled_output.shape)
      out, _ = self.lstm(last_hidden_state, None)
      #print(out.shape)
      #attn_output, attn_output_weights = self.attention(out, out, out)
      #print(attn_output.shape)
      #print(attn_output_weights.shape)
      NER = self.dropout(out)
      ACC = self.dropout(out[:, -1, :])
      #print(out.shape)
      output = self.fc(NER)
      out1 = self.fc1(ACC)
      
      return output, out1




class Bert_GRU(torch.nn.Module):
    def __init__(self, numclasses):
        super(Bert_GRU, self).__init__()
        self.numclasses = numclasses
        self.embeddim = embeddim
        self.numlayers = numlayers
        self.hiddendim_lstm = hiddendim_lstm
        self.dropout = torch.nn.Dropout(0.1)

        #self.lstm = torch.nn.LSTM(self.embeddim, self.hiddendim_lstm*2, bidirectional = True, batch_first=False)
        #self.conv = torch.nn.Conv1d( 100, self.hiddendim_lstm, 3, stride=2)
        self.bert = transformers.BertModel.from_pretrained('bert-base-uncased', return_dict=False)
        print("BERT Model Loaded")
        #self.lstm = torch.nn.LSTM(self.embeddim, self.hiddendim_lstm, bidirectional = True, batch_first=False)
        self.Gru = torch.nn.GRU(self.embeddim, self.hiddendim_lstm, bidirectional = False, batch_first=False) # noqa
        #self.attention = torch.nn.MultiheadAttention(self.hiddendim_lstm*2, num_heads = 2)
        self.fc = torch.nn.Linear(self.hiddendim_lstm, 4)
        self.fc1 = torch.nn.Linear(self.hiddendim_lstm, 5)

    def forward(self, ids, mask, token_type_ids):
      last_hidden_state, pooled_output = self.bert(ids, attention_mask = mask, token_type_ids = token_type_ids)
      #print(last_hidden_state.shape)
      #print(pooled_output.shape)
      out, _ = self.Gru(last_hidden_state, None)
      #print(out.shape)
      #attn_output, attn_output_weights = self.attention(out, out, out)
      #print(attn_output.shape)
      #print(attn_output_weights.shape)
      NER = self.dropout(out)
      ACC = self.dropout(out[:, -1, :])
      #print(out.shape)
      output = self.fc(NER)
      out1 = self.fc1(ACC)
      
      return output, out1



class Bert_Attention(torch.nn.Module):
    def __init__(self, numclasses, device):
        super(Bert_Attention, self).__init__()
        self.numclasses = numclasses
        self.embeddim = embeddim
        self.numlayers = numlayers
        self.hiddendim_lstm = hiddendim_lstm
        self.fchidden = fchidden
        self.dropout = torch.nn.Dropout(0.1)

        self.bert = transformers.BertModel.from_pretrained('bert-base-uncased', return_dict=False)
        print("BERT Model Loaded")
        self.Gru = torch.nn.GRU(self.embeddim, self.hiddendim_lstm, bidirectional = True, batch_first=False)

        q_t = np.random.normal(loc=0.0, scale=0.1, size=(1, self.hiddendim_lstm*2))
        self.q = torch.nn.Parameter(torch.from_numpy(q_t)).float().to(device)
        w_ht = np.random.normal(loc=0.0, scale=0.1, size=(self.hiddendim_lstm*2, self.fchidden)) # noqa
        self.w_h = torch.nn.Parameter(torch.from_numpy(w_ht)).float().to(device)

        self.fc = torch.nn.Linear(self.fchidden, 4)
        self.fc1 = torch.nn.Linear(self.fchidden, 5)


    def forward(self, inp_ids, att_mask, token_ids):
        last_hidden_state, pooler_output = self.bert(input_ids=inp_ids, attention_mask=att_mask, token_type_ids=token_ids)
        #print("1",last_hidden_state.shape)
        gru, _ = self.Gru(last_hidden_state, None)
        #print("2",out.shape)
        #out = self.attention(out)
        att = self.attention(gru)
        #print("3",out.shape)
        out = self.dropout(att)
        print("la",out.shape)
        output = self.fc(out)
        out1 = self.fc1(out)
        #print("la",out.shape)
        return output, out1

    def attention(self, h):
        #print("hhhhh",h.shape)
        v = torch.matmul(self.q, h.transpose(-2, -1)).squeeze(1)
        v = torch.nn.functional.softmax(v, -1)
        v_temp = torch.matmul(v.unsqueeze(1), h).transpose(-2, -1)
        #print("tem",v_temp.shape)
        v = torch.matmul(self.w_h.transpose(1, 0), v_temp).squeeze(2)
        #print(v.shape)
        return v


#model = Bert_Attention(numclasses, device)
#model = Bert_LSTM(numclasses)
#model = Bert_GRU(numclasses)
model = BERTClass()
model.to(device)
model

BERTClass(
  (l1): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    

In [ ]:

def loss_fn(output, target, mask, num_labels):
    lfn = torch.nn.CrossEntropyLoss()
    active_loss = mask.view(-1) == 1
    active_logits = output.view(-1, num_labels)[active_loss]
    #active_labels = torch.where(
    #    active_loss,
    #    target.view(-1),
    #    torch.tensor(lfn.ignore_index).type_as(target)
    #)
    active_labels = target.view(-1)[active_loss]
    #print(active_logits.shape)
    #print(active_labels.shape)
    loss = lfn(active_logits, active_labels)
    return loss
'''
import tensorflow as tf

def loss_fn(output, target, mask, num_labels):
  return tf.keras.losses.CategoricalCrossentropy()(output, target)
'''

def loss_fun(outputs, targets):
    #print(outputs.shape)
    #print(targets.shape)
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

In [ ]:
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [ ]:

def evaluate(loader, model, device):
    model.eval()
    with torch.no_grad():
        loss = 0.0
        y_pred = []
        y_true = []
        for _,data in enumerate(loader, 0):              
            #print(data)
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets_tag = data['target_tag'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)

            outputs, out = model(ids, mask, token_type_ids)
            loss1 = loss_fn(outputs, targets_tag, mask, num_tag)
            loss2 = loss_fun(out, targets)
            loss = loss1 + loss2
            preds = torch.sigmoid(out)
            y_pred.extend(preds.tolist())
            y_true.extend(targets.tolist())
            y_predict = np.array(y_pred) >= 0.5

        F1 = round((metrics.accuracy_score(y_true, y_predict)), 2) * 100
        return loss, F1

In [ ]:
def train(epoch):
    model.train()
    for _,data in enumerate(training_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets_tag = data['target_tag'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs, out = model(ids, mask, token_type_ids)
        #print(outputs.shape)
        #print(out.shape)
        optimizer.zero_grad()
        loss1 = loss_fn(outputs, targets_tag, mask, num_tag)
        loss2 = loss_fun(out, targets)
        loss = loss1 + loss2
        if _%5000==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}, Loss1:  {loss1.item()}, Loss2:  {loss2.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    #valloss, valacc = evaluate(dev_loader, model, device) # noqa

    #print(" Val Loss {} Val Acc {} ".format(valloss, valacc))

In [ ]:
for epoch in range(EPOCHS):
    train(epoch)

Epoch: 0, Loss:  2.219327688217163, Loss1:  1.5162782669067383, Loss2:  0.70304936170578
Epoch: 1, Loss:  0.4275926947593689, Loss1:  0.1389995664358139, Loss2:  0.2885931432247162
Epoch: 2, Loss:  0.27487969398498535, Loss1:  0.04333961755037308, Loss2:  0.23154006898403168
Epoch: 3, Loss:  0.10297857969999313, Loss1:  0.021696599200367928, Loss2:  0.08128198236227036
Epoch: 4, Loss:  0.1377234011888504, Loss1:  0.0146204624325037, Loss2:  0.12310294061899185


In [ ]:
 
def validation(epoch):
    model.eval()
    fin_targets=[]
    fin_targets_tag=[]
    fin_outputs=[]
    fin_out=[]
    with torch.no_grad():
        for _, data in enumerate(testing_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets_tag = data['target_tag'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs, out = model(ids, mask, token_type_ids)
            loss1 = loss_fn(outputs, targets_tag, mask, num_tag)
            loss2 = loss_fun(out, targets)
            loss = loss1 + loss2
            if _%5000==0:
                print(f'Epoch: {epoch}, Loss:  {loss.item()}, Loss1:  {loss1.item()}, Loss2:  {loss2.item()}')
            #outputs = torch.log(outputs)
            #outputs = torch.cat(outputs, dim=1).flatten()
            #print(outputs)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_targets_tag.extend(targets_tag.cpu().detach().numpy().tolist())
            fin_out.extend(torch.sigmoid(out).cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets, fin_targets_tag, fin_out

In [ ]:
best_accuracy = 0
predictions = []
truetarget = []
for epoch in range(EPOCHS):
    outputs, targets, targets_tag, out = validation(epoch)
    print(out[1])
    print(targets[1])
    for p, r in zip(outputs, targets_tag):
      #print(p)
      #print(r)
      outs = []
      tar = []
      for m, n in zip (p,r):
        if (n != 0):
          a = m.index(max(m))
          #print(a)
          tar.append(n)
          outs.append(a)
          #print(outs)
      predictions.append(outs)
      truetarget.append(tar)
        
    print(predictions[1])
    print(truetarget[1])
    print(outputs[1])
    print(targets_tag[1]) 
    out = np.array(out) >= 0.5
    #accuracy = metrics.accuracy_score(targets_tag, outputs)
    accuracy = metrics.accuracy_score(targets, out)
    f1_score_micro = metrics.f1_score(targets, out, average='micro')
    f1_score_macro = metrics.f1_score(targets, out, average='macro')
    #precision_score_micro = metrics.precision_score(targets, outputs, average='micro')
    precision_score_macro = metrics.precision_score(targets, out, average='macro')
    #recall_score_micro = metrics.recall_score(targets, outputs, average='micro')
    recall_score_macro = metrics.recall_score(targets, out, average='macro')
    print(f"Accuracy Score = {accuracy}")
    print(f"F1 Score (Micro) = {f1_score_micro}")
    print(f"F1 Score (Macro) = {f1_score_macro}")
    #print(f"precision_score (Micro) = {precision_score_micro}")
    print(f"precision_score (Macro) = {precision_score_macro}")
    #print(f"recall_score (Micro) = {recall_score_micro}")
    print(f"recall_score (Macro) = {recall_score_macro}")
    if accuracy > best_accuracy:
      torch.save(model.state_dict(), 'best_model_state.bin')
      best_accuracy = accuracy

Epoch: 0, Loss:  0.19558551907539368, Loss1:  0.17453227937221527, Loss2:  0.021053243428468704
[0.016304565593600273, 0.9943177103996277, 0.008073369972407818, 0.01605190522968769, 0.010720041580498219]
[0.0, 1.0, 0.0, 0.0, 0.0]
[1, 1, 1, 2, 3, 3, 1]
[1, 1, 1, 2, 3, 3, 1]
[[0.06589552015066147, 0.970349133014679, 0.19466933608055115, 0.32623490691185], [0.054947592318058014, 0.9954773783683777, 0.11391245573759079, 0.11239936202764511], [0.05344795063138008, 0.992094099521637, 0.10846182703971863, 0.1749168187379837], [0.04156026616692543, 0.36710864305496216, 0.9850741624832153, 0.24938958883285522], [0.09494815021753311, 0.08507059514522552, 0.38758379220962524, 0.993985116481781], [0.13764631748199463, 0.09246372431516647, 0.30599692463874817, 0.9953393936157227], [0.07195570319890976, 0.9959688186645508, 0.06147905811667442, 0.2725265324115753], [0.03206738829612732, 0.5744346976280212, 0.9738784432411194, 0.17113424837589264], [0.04712647572159767, 0.17607851326465607, 0.79034465

In [ ]:
from sklearn.metrics import classification_report
class_names = ['service', 'food', 'anecdotes/miscellaneous', 'price', 'ambiance']
print(classification_report(out, targets, target_names=class_names))

                         precision    recall  f1-score   support

                service       0.93      0.90      0.91       178
                   food       0.93      0.97      0.95       401
anecdotes/miscellaneous       0.77      0.91      0.84       198
                  price       0.87      0.95      0.91        76
               ambiance       0.87      0.89      0.88       114

              micro avg       0.88      0.93      0.91       967
              macro avg       0.87      0.92      0.90       967
           weighted avg       0.89      0.93      0.91       967
            samples avg       0.90      0.93      0.91       967



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
!pip install seqeval
from seqeval.metrics import f1_score, accuracy_score
from seqeval.metrics import classification_report

     |████████████████████████████████| 51kB 4.5MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp37-none-any.whl size=16172 sha256=74376b9ca8aa32052cc17609e04f9e96fb75a645d345b63a0a1189a924d5fb6c
  Stored in directory: /root/.cache/pip/wheels/52/df/1b/45d75646c37428f7e626214704a0e35bd3cfc32eda37e59e5f
Successfully built seqeval


In [ ]:
#print("F1-Score: {}".format(f1_score(pred, targ)))
print("Validation Accuracy: {}".format(accuracy_score(predictions, truetarget)))
pred = []
targ = []
for p in predictions:
  #print(p)
  q = []
  for p_i in p:
    #print(p_i)
    if p_i == 1:
      q.append('O')
      #print(p_i)
    elif p_i == 2:
      q.append('B')
      #print(p_i)
    else:
      q.append('I')
  pred.append(q)
    #print(predictions)
    #print(pred)
for p in truetarget:
  #print(p)
  q = []
  for p_i in p:
    #print(p_i)
    if p_i == 1:
      q.append('O')
      #print(p_i)
    elif p_i == 2:
      q.append('B')
      #print(p_i)
    else:
      q.append('I')
  targ.append(q)
print(pred)
print(targ)
class_names = ['Negative', 'Neutral', 'Positive']
print("F1-Score: {}".format(classification_report(pred, targ)))
print("F1-Score: {}".format(f1_score(pred, targ)))


print("Validation Accuracy: {}".format(accuracy_score(pred, targ)))

Validation Accuracy: 0.9701703588983907
[['O', 'B', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'B', 'I', 'I', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B', 'O', 'O', 'O', 'O', 'O', 'B', 'O', 'O', 'O', 'B', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'B', 'B', 'I', 'B', 'O', 'O', 'O', 'B', 'B', 'O', 'O', 'O', 'O', 'O', 'B', 'O'], ['O', 'O', 'O', 'B', 'I', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'B', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'B', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'B', 'B', 'O', 'O', 'O', 'O', 'O', 'B', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'B', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'B', 'B', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B', 'O', 'O',

In [ ]:
for i in range (0,50):
  print(predictions[i])
  print(truetarget[i])
  print(pred[i])
  print(targ[i])

[1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
['O', 'B', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['O', 'B', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
[1, 1, 1, 2, 3, 3, 1]
[1, 1, 1, 2, 3, 3, 1]
['O', 'O', 'O', 'B', 'I', 'I', 'O']
['O', 'O', 'O', 'B', 'I', 'I', 'O']
[1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1]
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B', 'O', 'O', 'O', 'O', 'O', 'B', 'O', 'O', 'O', 'B', 'O', 'O', 'O', 'O']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B', 'O', 'O', 'O', 'O', 'O', 'B', 'O', 'O', 'O', 'B', 'O', 'O', 'O', 'O']
[1, 1, 1, 2, 2, 3, 2, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 2, 1]
[1, 1, 2, 3, 3, 3, 3, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 2, 1]
['O', 'O', 'O', 'B', 'B', 'I', 'B', 'O', 'O', 'O', 'B', 'B', 'O', 'O', 'O', 'O', 'O', 'B', 'O']
['O', 'O', 'B', 'I', 'I', 'I', 'I', 'O', 'O', 'O', 'B', 'B', 'O', 'O', '